In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch


In [ ]:
def extract_keywords_bert(data):
    # 1. Load Tokenizer and Model
    model_name = "bert-base-cased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=3)  # O, B-KEY, I-KEY

    # 2. Preprocess Data (Only needed if fine-tuning)
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(examples["ad_line"], truncation=True, is_split_into_words=False)
        labels = []
        for i, label in enumerate(examples["labels"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            label_ids = []
            prev_word_idx = None
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)  # Special token, ignore loss
                elif word_idx != prev_word_idx:
                    label_ids.append(label[word_idx])  # Assign label
                else:
                    label_ids.append(-100)  # Subword of keyword, ignore loss
                prev_word_idx = word_idx
            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    def _extract_keywords(text):
        inputs = tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)

        predictions = torch.argmax(outputs.logits, dim=2)[0]
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

        keywords = []
        current_keyword = ""
        for token, prediction in zip(tokens, predictions):
            if prediction == 1:  # B-KEY
                if current_keyword:
                    keywords.append(current_keyword)
                current_keyword = token
            elif prediction == 2:  # I-KEY
                current_keyword += token.replace("##", "")
            elif current_keyword:
                keywords.append(current_keyword)
                current_keyword = ""
        if current_keyword:
            keywords.append(current_keyword)
        return keywords

    data['bert_keywords'] = data['ad_line'].apply(_extract_keywords)
    return data


In [ ]:
data = pd.DataFrame({'ad_line': [
    "Buy the best running shoes online",
    "Affordable wireless headphones",
    "Watch 4k smart TV sale"
]})

In [ ]:
# Extract keywords using BERT
data = extract_keywords_bert(data)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(data[['ad_line', 'bert_keywords']])

                             ad_line  \
0  Buy the best running shoes online   
1     Affordable wireless headphones   
2             Watch 4k smart TV sale   

                                     bert_keywords  
0                [thebest, running, shoes, online]  
1  [##ff, ##ord, ##able, wireless, head, ##phones]  
2                             [4, smart, TV, sale]  
